In [2]:

import pandas as pd
import latentscope as ls
import h5py
# !pip install datasets
from datasets import load_dataset

In [3]:
# TODO: show downloading sample from fineweb

In [10]:
with h5py.File("/Users/enjalot/latent-scope-data/fineweb-edu-10k/saes/sae-001.h5", "r") as f:
    print(f.keys())
    top_acts = f.get("top_acts")[:]
    top_indices = f.get("top_indices")[:]
    print(top_acts.shape)
    print(top_indices.shape)


<KeysViewHDF5 ['top_acts', 'top_indices']>
(10000, 64)
(10000, 64)


In [11]:
top_acts[0]

array([0.21127602, 0.14851032, 0.13167411, 0.12901936, 0.12542538,
       0.12050804, 0.11599681, 0.10911667, 0.10245717, 0.09247096,
       0.08518378, 0.08365664, 0.07800631, 0.06706782, 0.06647921,
       0.05952992, 0.05913502, 0.05650827, 0.05593583, 0.04889879,
       0.04714418, 0.04397228, 0.04274654, 0.04017923, 0.04017694,
       0.03835431, 0.03787563, 0.03787341, 0.03650027, 0.03406205,
       0.03383416, 0.03364984, 0.0322901 , 0.03211844, 0.03160176,
       0.03152224, 0.03134899, 0.02972465, 0.02833397, 0.02795491,
       0.02785561, 0.02775786, 0.0272976 , 0.02706236, 0.02680814,
       0.02642737, 0.02591424, 0.02473864, 0.02473825, 0.02321817,
       0.02317835, 0.0230809 , 0.02284331, 0.02217324, 0.0219964 ,
       0.02166185, 0.02163344, 0.02138862, 0.0213408 , 0.02086308,
       0.02070821, 0.02066883, 0.02055968, 0.02004253], dtype=float32)

In [12]:
top_indices[0]

array([ 8944,  7583, 16225, 14943, 11176, 23917,  4919,  4941,  4000,
       10367,  5983,  7627,  9398, 16267, 11963, 16708,  7518, 18204,
        9535,  2724, 18167,  2228, 19213, 18010, 22989,  5432, 10935,
       22337,  4004, 11119,  7774, 17695, 10567, 21551, 23250,  1460,
       22268,  9566, 18271, 21744, 21539, 10463, 20740,  5340, 10539,
        9217, 15487,  7070, 22817, 17483, 24236, 18312, 20104, 12048,
        3322,    38,  4430, 13258,  3523,  1725, 20912, 15362,  8929,
         519])

In [1]:
# experiment with loading a saved UMAP model for the SAE top activations that we can use to map any dataset
from huggingface_hub import hf_hub_download
import pickle
import json

# Download the UMAP model from HuggingFace Hub
model_path = hf_hub_download(
    repo_id="enjalot/sae-nomic-text-v1.5-FineWeb-edu-100BT",
    filename="umap/64_32/umap-top10.pkl"
)
# Download the metadata JSON file
metadata_path = hf_hub_download(
    repo_id="enjalot/sae-nomic-text-v1.5-FineWeb-edu-100BT",
    filename="umap/64_32/umap-top10-metadata.json"
)


# Load the UMAP model
with open(model_path, 'rb') as f:
    umap_model = pickle.load(f)


# Load the metadata
with open(metadata_path, 'r') as f:
    umap_metadata = json.load(f)

print("UMAP model loaded:", umap_model)
print("metadata", umap_metadata)


umap-top10.pkl:   0%|          | 0.00/287M [00:00<?, ?B/s]

umap/64_32/umap-top10-metadata.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

Fri Nov  8 11:04:27 2024 Building and compiling search function
UMAP model loaded: UMAP(angular_rp_forest=True, metric='cosine', n_jobs=1, n_neighbors=25, random_state=42, verbose=True)
metadata {'umap_params': {'n_neighbors': 25, 'min_dist': 0.1, 'metric': 'cosine', 'n_components': 2, 'random_state': 42}, 'bounds': {'x_min': -0.5994952321052551, 'x_max': 9.158385276794434, 'y_min': -0.24440008401870728, 'y_max': 7.851518630981445}}


In [ ]:
# TODO: load some embeddings and fit this
xy = umap_model.transform(embeddings)

In [ ]:
min_values = np.array([umap_metadata['bounds']["x_min"], umap_metadata['bounds']["y_min"]])
max_values = np.array([umap_metadata['bounds']["x_max"], umap_metadata['bounds']["y_max"]])
normalized_points = (xy - min_values) / (max_values - min_values)
normalized_points = 2 * normalized_points - 1
# you can now get the x,y coordinates that are scaled to the original umap x,y
# this means any dataset projected in this way will overlap in the exact same space as the original SAE top activations

# results_df["x"] = normalized_points[:, 0]
# results_df["y"] = normalized_points[:, 1]
# results_df.head()